In [ ]:
from catalogos_capes import datasets

df = datasets.get_all_datasets_with_resources()
df = datasets.get_dataset_by_name(df)

In [ ]:
from tqdm.auto import tqdm
from typing import Iterable

def download_files(urls: Iterable[str], dest_dir: str):
    """Download multiple files to the given directory."""
    files = []
    for url in tqdm(urls, desc="Baixando..."):
        files.append(datasets.download_file(url, dest_dir))
    return files

In [ ]:
[df['url'].iloc[-2]]

In [ ]:
download_files([df['url'].iloc[-2]], "./data")

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
from pandas.errors import ParserError

dataset = "../BD_Chatbot/data/catalogo-capes/"
files = list(Path(dataset).glob("**/*.csv"))

with sqlite3.connect("capes.sqlite") as conn:
    cur = conn.cursor()
    for filepath in tqdm(files):
        try:
            data = pd.read_csv(filepath, sep=';', encoding="latin-1", low_memory=False)
        except ParserError:
            data = pd.read_csv(filepath, sep=',', encoding="latin-1", low_memory=False)
        data.columns = (
            data.columns.str.lower()
            .str.normalize("NFKD")
            .str.encode("ascii", errors="ignore")
            .str.decode("utf-8")
            .str.replace(r"\s+", " ", regex=True)
            .str.replace(r"[^a-z0-9]", "_", regex=True)
            .str.strip("_")
        )

        # create a table from the dataframe
        data.to_sql(filepath.stem, conn, if_exists="replace", index=False)

        # commit the changes to db
        conn.commit()